In [3]:
import random

In [4]:
def gen_super_crec(n_terms):
    '''Genera una sucesion supercreciente aleatoria
    Devuelve la sucesion generada
    
    Args:
        n_terms (int): numero de terminos que tendra la sucesion
        
    Returns:
        ssc (lista): lista que contiene los numeros de la sucesion supercreciente
    '''
    ssc = []
    last = random.randint(1,10)
    ssc.append(last)
    for i in range(1,n_terms):
        last = last+random.randint(last,last+10)
        ssc.append(last)
        
    return ssc

def multiplier(mod, mult_ini):
    i = mult_ini+1
    while(True):
        if(mcd(i,mod) == 1):
            return i
        i+=1

def inverse(p, mod):
    '''Funcion que calcula el inverso multiplicativo de p en Zmod
    Esta funcion utiliza el algoritmo de Euclides extendido con el cual calculamos:
                            1 = u*mod+v*p
    donde v es el inverso multiplicativo de p en Zmod.
    Devuelve el inverso multiplicativo en caso de que exista, y -1 si no existe
    
    Args:
        p (int): numero del que obtener el inverso multiplicativo
        mod (int): modulo en el que calcular el inverso multiplicativo
    
    Returns:
        vi (int): inverso multiplicativo de p en Zmod, -1 si no existe    
    '''
    # si el mcd no es uno, no tiene inv mult en Zmod
    if(mcd(p,mod) != 1):
        return -1
    
    r0 = mod # r(i-1)
    ri = p # r(i)
    u0 = 1 # u(i-1)
    v0 = 0 # v(i-1)
    ui = 0 # u(i)
    vi = 1 # v(i)
    
    # cuando ri=1 ya tenemos en vi el inv multiplicativo
    while(ri != 1):
        #calculamos el r(i+1)
        r2 = r0 % ri
        # y el qi
        qi = r0 // ri
        #calculamos los multiplicadores u y v para esta ronda
        u2 = u0-(qi*ui)
        v2 = v0-(qi*vi)
        
        #actualizamos los valores
        r0 = ri
        ri = r2
        u0 = ui
        ui = u2
        v0 = vi
        vi = v2
    
    return int(vi%mod)

def mod_mult_inv(l_sc):
    '''Funcion que calcula un multiplicador, su inverso y el modulo de la aritmetica en funcion
    a una clave privada
    Devuelve los tres valores mencionados.
    
    Args:
        l_sc (lista): lista que contiene la clave privada
    
    Returns:
        mul (int): multiplicador cuyo valor va entre 1 y mod/2
        inv (int): inverso multiplicativo de mul en Zmod
        mod (int): aritmetica a utilizar
        
    '''
    mod = sum(l_sc)+1
    mul = multiplier(mod,random.randint(1,int(mod/2)))
    inv = inverse(mul,mod)
    return mul,inv,mod
    
def gen_sucesion_publica(l_sc, p, mod):
    '''Funcion que calcula una clave publica en funcion a una privada, un multiplicador y un modulo
    Devuelve la clave privada
    
    Args:
        l_sc (lista): lista que contiene la clave privada
        p (int): multiplicador
        mod (int): aritmetica a utilizar 
    
    Returns:
        l_pub (lista): clave publica de la clave privada recibida calculada
        
    '''
    l_pub = []
    for elemento in l_sc:
        l_pub.append((p*elemento) % mod)
    return l_pub

def l_publica_2_l_super_crec(l_pub, q, mod):
    '''Funcion que calcula una clave privada en funcion a una publica, un inverso y un modulo
    Devuelve la clave privada
    
    Args:
        l_pub (lista): lista que contiene la clave publica
        q (int): inverso
        mod (int): aritmetica a utilizar 
    
    Returns:
        l_sc (lista): clave privada calculada (lista supercreciente)
        
    '''
    l_sc = []
    for elemento in l_pub:
        l_sc.append(((q*elemento) % mod))
    return l_sc

def mcd(a,b):
    '''Calcula el maximo comun divisor entre a y b utilizando el algoritmo de euclides
    Devuelve el maximo comun divisor entre ambos numeros
    
    Args:
        a (int): primer numero
        b (int): segundo numero
        
    Returns:
        mcd (int): maximo comun divisor entre a y b
    '''
    if(a<b):
        a_aux = a
        a = b
        b = a_aux
        
    if((a % b) == 0):
        return b
    return mcd(b,a%b)
    
    

In [8]:
def gen_random_bit_list(n_bits):
    '''Funcion que genera una lista aleatoria de bits
    
    Args:
        n_bits (int): numero de bits que contendra la lista generada 
    
    Returns:
        l_bits (lista): lista de bits generada
        
    '''
    l_bits = []
    for i in range(0,n_bits):
        l_bits.append(random.choice([0,1]))
    return l_bits

def mh_encrypt(l_bits, l_pub, mod):
    '''Funcion que cifra una cadena de bits utilizando una clave publica y el modulo
    
    Args:
         l_bits (lista): lista de bits a cifrar
         l_pub (lista): lista que contiene la clave publica
         mod (int): aritmetica a utilizar 
    
    Returns:
        encrypted_blocks (lista): lista de enteros donde cada entero contiene un bloque cifrado
    '''
    encrypted_blocks = []
    block_size = len(l_pub)
    #Anadimos 0s al final hasta que sea de la longitud de l_pub
    while((len(l_bits) % block_size) != 0):
        l_bits.append(0)
    for i in range(0,len(l_bits),block_size):
        res = 0
        k=0
        for ki in l_pub:
            res+=ki * l_bits[i+k]
            k+=1
        encrypted_blocks.append(res)
        
    return encrypted_blocks

def mh_block_decrypt(c, l_sc, inv, mod):
    '''Funcion que descifra un entero utilizando una clave privada, un inverso y el modulo
    
    Args:
         c (int): entero que contiene un bloque cifrado
         l_sc (lista): lista supercreciente con la clave privada
         inv (int): inverso
         mod (int): aritmetica a utilizar 
    
    Returns:
        decrypted_block (lista): lista de bits con el bloque descifrado
    '''
    
    # creamos una lista vacia con el tamano de la clave privada
    decrypted_block = [None] * len(l_sc)
    # guardamos en la variable resto el valor del bloque codificado * inverso modulo mod
    resto = (c*inv) % mod
    
    # creamos el diccionario numMapper que mapea los valores de la clave privada con su indice
    # para despues ir anadiendo los bits al bloque decodificado de forma ordenada
    numMapper = {}
    i=0
    for num in l_sc:
        numMapper[num] = i
        i+=1
 
    # aplicamos el problema de la suma en la clave publica ordenada de mayor a menor
    for ki in sorted(l_sc,reverse=True):
        if(resto>=ki):
            # si el numero entra, le anadimos un uno al bloque
            decrypted_block[numMapper[ki]] = 1
            # y actualizamos el resto con el numero restante
            resto -= ki
        else:
            # si no entra el numero en el numero estudiado, anadimos un cero y continuamos
            decrypted_block[numMapper[ki]] = 0
    
    return decrypted_block


def mh_decrypt(l_cifra, l_sc, inv, mod):
    '''Funcion que descifra un texto cifrado utilizando una clave privada, un inverso y el modulo
    
    Args:
         l_cifra: lista que contiene los bloques cifrados (texto completo)
         l_sc (lista): lista supercreciente con la clave privada
         inv (int): inverso
         mod (int): aritmetica a utilizar 
    
    Returns:
        blocks (lista): lista de bits con el texto descifrado
    '''
    blocks = []
    for c in l_cifra:
        blocks += mh_block_decrypt(c, l_sc, inv, mod)
        
    return blocks

In [9]:
for i in range(1,100):
    ssc = gen_super_crec(i*10)
    mul,inv,mod = mod_mult_inv(ssc)
    assert((mul*inv) % mod == 1)
    
    pub = gen_sucesion_publica(ssc,mul,mod)
    texto = gen_random_bit_list(random.randint(i*10,i*20))
    cifrado = mh_encrypt(texto,pub,mod)
    desc = mh_decrypt(cifrado, ssc, inv,mod)
    assert(texto == desc[0:len(texto)])
    print('Test with list size',i, 'OK')

Test with list size 1 OK
Test with list size 2 OK
Test with list size 3 OK
Test with list size 4 OK
Test with list size 5 OK
Test with list size 6 OK
Test with list size 7 OK
Test with list size 8 OK
Test with list size 9 OK
Test with list size 10 OK
Test with list size 11 OK
Test with list size 12 OK
Test with list size 13 OK
Test with list size 14 OK
Test with list size 15 OK
Test with list size 16 OK
Test with list size 17 OK
Test with list size 18 OK
Test with list size 19 OK
Test with list size 20 OK
Test with list size 21 OK
Test with list size 22 OK
Test with list size 23 OK
Test with list size 24 OK
Test with list size 25 OK
Test with list size 26 OK
Test with list size 27 OK
Test with list size 28 OK
Test with list size 29 OK
Test with list size 30 OK
Test with list size 31 OK
Test with list size 32 OK
Test with list size 33 OK
Test with list size 34 OK
Test with list size 35 OK
Test with list size 36 OK
Test with list size 37 OK
Test with list size 38 OK
Test with list size 3

# Cuestiones sobre el cifrado Merkle-Hellman

1. Dado el posible tamaño de los términos de la sucesión supercreciente, es necesario trabajar con enteros de tamaño adecuado. Averiguar el tamano máximo de un entero en Python.
En la versión 3 de Python (https://docs.python.org/3.1/whatsnew/3.0.html#integers), se eliminó la limitación máxima para números enteros, y a efectos prácticos, esta no existe.
En la versión 2 de Python, este límite máximo sí que existía, y se podía obtener ejecutando:
```python
>>> sys.maxint
9223372036854775807
```
Y como se puede observar, este máximo era 9223372036854775807. Esta función ha sido deshabilitada en la versión 3, pero en caso de necesitar este número por cuestiones de compatibilidad entre versiones o migraciones a la versión 3, se puede obtener con el comando:
```python
>>> sys.maxsize
9223372036854775807
```

2. Un elemento importante en el algoritmo Merkle–Hellman es la longitud de las sucesiones empleadas, lo que a su vez influye en el valor maximo de sucesión supercreciente y el módulo. Si dicha sucesión tiene N terminos, estimar los valores mínimos del ultimo término de una sucesión supercreciente de N terminos y del módulo. Sugerencia: considerar el ejemplo de la sucesion sn = 2^n, n = 0, 1, 2, ...
Contando con que la sucesión sigue una distribución 2^n para un N arbitrario, el límite mínimo del último término obviamente será 2^(N-1). Estudiando cada uno de los números, podemos observar que cada número, al ser potencia de 2 representa un bit nuevo en una representación binaria (ejemplo: `N=3 => Sn = [b1, b10, b100]`). Teniendo esto en cuenta, el módulo, cuyo mínimo debería ser la suma de todos los números existentes en la lista, tendrá como mínimo un número cuya representación binaria tiene N unos + 1, lo cual es igual a 2^N.

3. A la vista de las dos cuestiones previas, discutir cual puede ser la longitud máxima razonable de la sucesión supercreciente.
Considerando la capacidad de cómputo actual, en la cual las recomendaciones para algoritmos de clave pública como RSA son utilizar claves de 4096 bits, la longitud de lista recomendada es la cual hace que el módulo tenga 4096 bits. Dicha longitud es 12 (= log2(4096)) ya que de esa forma el módulo tendrá como mínimo 4096 bits.

4. Un enfoque trivial para la función inverse(p, mod) es probar con enteros de manera iterada hasta encontrar un q tal que p * q % mod == 1. Sin embargo, esto es muy costoso computacionalmente y se puede mejorar mediante una variante del algoritmo de Euclides. Describir aquí dicha variante y estimar su coste computacional.
Hemos aplicado el algoritmo de Euclides extendido para obtener el inverso multiplicativo en la práctica. Este algoritmo utiliza el algoritmo de Euclides de forma secuencial con un número entero y un módulo hasta tener la siguiente fórmula:
```
1 = u*mod+v*p % (mod)
```
Dónde v es el inverso multiplicativo de p en Zmod.
Teniendo en cuenta que cada iteración tiene un coste de O(1) y que calculamos tantas iteraciones como O(log N) (porque en cada iteración se le aplica el módulo al número siguiente), entonces el coste computacional del algoritmo de Euclides es de O(log N).